In [2]:
#from pytorch_forecasting.data.examples import generate_ar_data
import matplotlib.pyplot as plt
import pandas as pd
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
import pytorch_lightning as pl
from pytorch_forecasting import NegativeBinomialDistributionLoss, DeepAR
import torch
from pytorch_forecasting.data.encoders import TorchNormalizer
import numpy as np


In [3]:
df_np = np.load('2_freq_stoch_nbinom_dem.npy')

df = pd.DataFrame()
df['series'] = np.zeros((len(df_np))).astype(int)
df['time_idx'] = np.arange(0,len(df_np))
df['value'] = df_np


df.to_csv('2_freq_stoch_nbinom_dem.csv')
data = df

In [4]:
# data = pd.read_csv('1_f_nbinom_train.csv')
# data

In [5]:
data["date"] = pd.Timestamp("2021-09-24") + pd.to_timedelta(data.time_idx, "H")


data['_hour_of_day'] = data["date"].dt.hour#.astype(str)
data['_day_of_week'] = data["date"].dt.dayofweek#.astype(str)
data['_day_of_month'] = data["date"].dt.day#.astype(str)
data['_day_of_year'] = data["date"].dt.dayofyear#.astype(str)
data['_week_of_year'] = data["date"].dt.weekofyear#.astype(str)
data['_month_of_year'] = data["date"].dt.month#.astype(str)
data['_year'] = data["date"].dt.year#.astype(str)

data['value'] = data['value'].astype(float)
print(type(data['value'][0])) 
print(len(data.iloc[0:-620]))



<class 'numpy.float64'>
4980


C:\Users\josepeeterson.er\AppData\Local\Temp\ipykernel_3356\357397288.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data['_week_of_year'] = data["date"].dt.weekofyear#.astype(str)


In [6]:
max_encoder_length = 60
max_prediction_length = 20
training_cutoff = data["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    data.iloc[0:-620],
    time_idx="time_idx",
    target="value",
    # categorical_encoders={"series": NaNLabelEncoder(add_nan=True).fit(data.series), "_hour_of_day": NaNLabelEncoder(add_nan=True).fit(data._hour_of_day), \
    #    "_day_of_week": NaNLabelEncoder(add_nan=True).fit(data._day_of_week), "_day_of_month" : NaNLabelEncoder(add_nan=True).fit(data._day_of_month), "_day_of_year" : NaNLabelEncoder(add_nan=True).fit(data._day_of_year), \
    #     "_week_of_year": NaNLabelEncoder(add_nan=True).fit(data._week_of_year), "_month_of_year": NaNLabelEncoder(add_nan=True).fit(data._month_of_year) ,"_year": NaNLabelEncoder(add_nan=True).fit(data._year)},
    group_ids=["series"],
    min_encoder_length=max_encoder_length,
    max_encoder_length=max_encoder_length,
    min_prediction_length=max_prediction_length,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=["value"],
    time_varying_known_reals=["_hour_of_day","_day_of_week","_day_of_month","_day_of_year","_week_of_year","_month_of_year","_year" ],
    add_relative_time_idx=False,
    randomize_length=None,
    scalers={},
    target_normalizer=TorchNormalizer(method="identity",center=False,transformation=None )

)

In [7]:


#training.data['target'][0][0:4]

In [8]:
validation = TimeSeriesDataSet.from_dataset(
    training,
    data.iloc[-620:-420],
    # predict=True,
    stop_randomization=True,
)       

In [9]:
batch_size = 2
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)


In [10]:
# i=0
# for x,y in train_dataloader:
#     print('xen',x["encoder_target"])
#     print('xen_cont',x['encoder_cont'])
#     print('xde',x["decoder_target"])
#     print('xde_cont',x["decoder_cont"])
#     print('y',y[0])
#     i+=1
#     if i == 2:
#         break

In [11]:
# save datasets
training.save("training.pkl")
validation.save("validation.pkl")

In [12]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=5, verbose=False, mode="min")
lr_logger = LearningRateMonitor()


In [ ]:
trainer = pl.Trainer(
    max_epochs=1,
    gpus=0,
    gradient_clip_val=0.1,
    limit_train_batches=30,
    limit_val_batches=3,
    # fast_dev_run=True,
    # logger=logger,
    # profiler=True,
    callbacks=[lr_logger, early_stop_callback],
)


In [ ]:
deepar = DeepAR.from_dataset(
    training,
    learning_rate=0.1,
    hidden_size=32,
    dropout=0.1,
    loss=NegativeBinomialDistributionLoss(),
    log_interval=10,
    log_val_interval=3,
    # reduce_on_plateau_patience=3,
)
print(f"Number of parameters in network: {deepar.size()/1e3:.1f}k")


In [15]:
# #find optimal learning rate
# deepar.hparams.log_interval = -1
# deepar.hparams.log_val_interval = -1
# trainer.limit_train_batches = 1.0
# res = trainer.tuner.lr_find(
#     deepar, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader, min_lr=1e-5, max_lr=1e2
# )

In [ ]:
torch.set_num_threads(10)
trainer.fit(
    deepar,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [17]:
# calcualte mean absolute error on validation set
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])

print(actuals.shape)

predictions = deepar.predict(data=val_dataloader,mode='prediction',return_index=True,num_workers=8,show_progress_bar=True)



#print(f"Mean absolute error of model: {(actuals - torch.tensor(predictions)).abs().mean()}")


torch.Size([121, 20])


Predict:   0%|          | 0/61 [00:00<?, ? batches/s]

In [18]:
print(actuals)
print((predictions[0][0:10]))



tensor([[ 8.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  2.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0., 14.,  ...,  0.,  7.,  0.],
        [ 0., 14.,  0.,  ...,  7.,  0.,  0.],
        [14.,  0.,  0.,  ...,  0.,  0.,  0.]])
tensor([[1.3500, 1.0800, 1.5500, 2.0700, 1.5800, 1.7300, 1.4800, 1.1000, 1.1500,
         1.4800, 1.4700, 1.6100, 0.8600, 1.3600, 1.0300, 1.4300, 1.1900, 1.3900,
         1.3500, 1.4000],
        [1.8500, 1.0100, 1.2000, 0.9100, 1.2000, 1.1100, 1.2400, 1.2500, 1.3100,
         1.1800, 1.6800, 1.0200, 1.3800, 1.2000, 2.0500, 1.3300, 1.4900, 1.1700,
         1.4500, 1.4800],
        [1.0800, 1.2500, 1.4600, 0.9600, 1.5600, 1.4000, 1.1400, 1.4800, 1.7100,
         1.4200, 2.1100, 1.4100, 1.1300, 1.5100, 1.3500, 1.6100, 1.4800, 1.7200,
         1.4000, 1.6600],
        [1.6200, 1.6300, 1.6900, 1.4900, 1.4800, 1.1900, 1.4700, 1.7900, 1.3000,
         0.9700, 1.6200, 1.3200, 1.4300, 1.3900, 0.9600, 1.6100, 